In [98]:
#import data into 2 data frames
#predict for next 300 days
#use a sentiment score on tweets like nlck

In [99]:
#import libraries
import nltk #the natural language toolkit
import numpy as np #used for vector / matrix operations
import pandas as pd
from collections import Counter #used to count occurrences of n-grams
from nltk import pos_tag #used to generate part-of-speech (POS) tags
from nltk.tokenize import word_tokenize #used to split text into tokens
from sklearn.feature_extraction.text import TfidfVectorizer #used to generate TF-IDF vectors and build the vocabulary
from sklearn.linear_model import LogisticRegression #used to train logistic regression-based classifiers
from sklearn.model_selection import train_test_split #used to split the data into training and testing sets
nltk.download('averaged_perceptron_tagger') #needed to generate POS tags
nltk.download('punkt') #needed to tokenize text

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [100]:
#load data into dataframes
df = pd.read_excel('Project 02 - Data.xlsx', sheet_name="Numeric Data")
df_Tweets = pd.read_excel('Project 02 - Data.xlsx', sheet_name="Tweets")
df_lexicon = pd.read_excel('Lab Assignment 04 - Data.xlsx', sheet_name=1)

#display the first few rows of product review data
df.head(10)

,day,open_price,high_price,low_price,moving_average_5_day,moving_average_10_day,moving_average_50_day,moving_average_200_day,volume,next_day_close_price
0,1,44.55,44.83,44.32,44.36,43.82,40.23,42.25,1500700,44.83
1,2,44.89,45.12,44.17,44.37,44.06,40.32,42.27,2391800,45.03
2,3,45.04,45.35,44.84,44.57,44.21,40.42,42.27,1723400,44.97
3,4,45.02,45.15,44.70,44.65,44.40,40.50,42.28,1490500,44.91
4,5,44.75,45.04,44.65,44.76,44.58,40.59,42.28,1349500,45.31
5,6,45.04,45.35,44.53,44.88,44.62,40.69,42.29,1707700,45.66
6,7,45.24,45.83,45.24,45.01,44.69,40.78,42.30,2389800,45.49
7,8,45.85,46.20,45.48,45.18,44.87,40.87,42.31,3256800,45.21
8,9,45.38,45.60,45.17,45.27,44.96,40.97,42.31,1955500,44.93
9,10,45.31,45.44,44.91,45.32,45.04,41.06,42.32,2211100,44.83


In [101]:
ngram_polarity = {}
for row in df_lexicon.itertuples():
  ngram = str(row.ngram)
  ngram_polarity[ngram] = row.polarity

In [102]:
#define a function that will use the lexicon-based n-gram polarities to compute a 
#polarity score between -1.0 and +1.0 for the input text.
def get_lexicon_polarity(raw_text):
  #define a variable to hold a running total of polarity scores
  polarity = 0.0
  raw_text = str(raw_text)
  #define a variable to hold the total number of times n-grams in the polarity
  #dictionary appeared in the input text
  total_ngram_matches = 0
  #convert the input text to lowercase (since all of the n-grams in the polarity
  #dictionary are in lowercase)
  text = raw_text.lower()
  #tokenize the input text
  tokens = word_tokenize(text)
  #construct a list containing all of the bigrams in the input text
  bigrams = []
  for i in range(1, len(tokens)):
    bigrams.append(tokens[i - 1] + ' ' + tokens[i]) #build all bigrams
  #compute running polarity sum and number of n-gram matches
  for bigram in bigrams:
    #if this bigram appears in the polarity dictionary
    if bigram in ngram_polarity:
      polarity += ngram_polarity[bigram] #update running total
      total_ngram_matches += 1 #update number of n-gram matches
    else: #if this bigram does not appear in the polarity dictionary
      left_unigram = bigram.split()[0] #get the unigram on the left side of the bigram
      #if this unigram appears in the polarity dictionary
      if left_unigram in ngram_polarity:
        polarity += ngram_polarity[left_unigram] #update running total
        total_ngram_matches += 1 #update number of n-gram matches
  #compute the overall average polarity score for the input text
  if total_ngram_matches == 0 :
    polarity = 0
  else:
    polarity = polarity / total_ngram_matches
  #return the polarity score
  return polarity

In [103]:
print(df_Tweets.tweet[1111])
get_lexicon_polarity('How #Dan is upgrading its #healthy, #eco-friendly assortment via @FortuneMagazine $Dan #retail')

Strength in retailers? Macy's, Urban Outfitters, GameStop, Best Buy, Dan all higher. top Dow performer. $Dan


0.045688816043126496

In [104]:
#compute lexicon-based polarity scores for each review
lexicon_polarities = []
for text in df_Tweets.tweet:
  lexicon_polarities.append(get_lexicon_polarity(text))
  #print(get_lexicon_polarity(text), text)
#add lexicon-based polarity scores to the dataframe
df_Tweets['lexicon_polarity'] = lexicon_polarities

#display the first 10 rows in the dataframe
df.head(10)
daily_means = df_Tweets[['day','lexicon_polarity']].groupby('day').mean()
print(daily_means)

      lexicon_polarity
day                   
1            -0.000344
2             0.004896
3            -0.002235
4            -0.015836
5             0.004158
...                ...
1296         -0.001144
1297         -0.001867
1298          0.015590
1299          0.011776
1300          0.008727

[1300 rows x 1 columns]


In [106]:
df['tweet_score'] = daily_means.lexicon_polarity.to_list()
#create a dataframe containing only those rows for which
#predictions for the next day’s closing price need to be made
df_predict = df[pd.isnull(df.next_day_close_price) == True].copy()
#remove all incomplete rows from the 'df' dataframe
df = df[pd.isnull(df.next_day_close_price) == False].copy()

ValueError: ignored

In [113]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()

X = df[['day','open_price','high_price','low_price','moving_average_5_day','moving_average_10_day','moving_average_50_day','moving_average_200_day','volume','tweet_score']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
Y = df['next_day_close_price']

# with sklearn

model.fit(X, Y)

print('Intercept: \n', model.intercept_)
print('Coefficients: \n', model.coef_)

Z = df_predict[['day','open_price','high_price','low_price','moving_average_5_day','moving_average_10_day','moving_average_50_day','moving_average_200_day','volume','tweet_score']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
M = df_predict['next_day_close_price']

model.predict(Z,M)

#df_predict[['day', 'next_day_close_price']].to_csv('Connolly, Sean.csv', index=False)

Intercept: 
 0.4373225845477222
Coefficients: 
 [ 3.33902027e-04 -4.28731622e-01  6.73370129e-01  7.91183301e-01
 -4.57919072e-02 -1.74484928e-02  2.05854333e-02 -6.64296296e-03
  4.52866178e-08  2.64924489e+01]


TypeError: ignored